In [73]:
import cv2
import numpy as np
from math import *

undistorted_image = cv2.imread("000001.jpg")
undistorted_image = cv2.resize(undistorted_image, (undistorted_image.shape[1], undistorted_image.shape[0]), interpolation=cv2.INTER_CUBIC)
undistorted_image.shape

(900, 1600, 3)

In [74]:
def PointMap(xx, yy, r):
    ''' 
    将一个柱面图像上的点用等距模型映射到单位柱面
    input:
        xx, yy: 输入参数，畸变图像上的点的坐标
        theta_max 柱面镜头视场角
        r  制作的柱面图像半径
    output:
        new_x, new_y : 输出参数，单位球面上点的坐标
    '''
    f = r
    tx = f * np.tan(xx / f)
    ty = yy / np.cos(xx / f)

    return tx, ty


In [75]:

def RectifyMap(r, ori_height, ori_width):
    '''
    生成从原图像到柱面图像的坐标的映射矩阵mapx mapy  
    input:
	    r: 圆半径，柱面图像半径
    output:
        mapy, mapy: 映射关系
    '''
    height = ori_height   # 映射图像的高度
    width = int(r * np.arctan(ori_width / 2 / r)) * 2 # 使用柱面参数计算映射图像宽度

    center_x = width / 2
    center_y = height / 2

    x = np.linspace(0, width-1, num=width, dtype=int)
    y = np.linspace(0, height-1, num=height, dtype=int)
    centx = np.ones_like(x, dtype=int) * center_x
    centy = np.ones_like(y, dtype=int) * center_y
    x = x - centx
    y = centy - y
    xx, yy = np.meshgrid(x, y)

    mapx, mapy = PointMap(xx, yy, r)
    
    return mapx, mapy

ori_height, ori_width = undistorted_image.shape[0:2]
mapx, mapy = RectifyMap(1000, ori_height, ori_width)

In [76]:
def UndisImage(undistorted_image, mapx, mapy):
    '''
    矫正图像
    '''
    assert(mapx.shape == mapy.shape)
    height, width = undistorted_image.shape[0:2]
    cx = width / 2
    cy = height / 2

    x = (mapx + np.ones_like(mapx) * cx)
    y = (np.ones_like(mapy) * cy - mapy)
    
    distort_image = cv2.remap(undistorted_image, x.astype(np.float32), y.astype(np.float32), cv2.INTER_LINEAR)

    return distort_image

distort_image = UndisImage(undistorted_image, mapx, mapy)

cv2.imwrite('cylindrical.jpg', distort_image)

True